In [52]:
import numpy as np
from numpy import asarray
import cv2
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data


In [53]:
class Conv_layer:
    def __init__(self,num_filters,filter_size):
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.conv_filter = np.random.randn(num_filters, filter_size, filter_size)/(filter_size*filter_size)
        
    def image_region(self,image):
        height,width = image.shape
        self.image = image
        for j in range(height - self.filter_size + 1):
            for k in range(width - self.filter_size + 1):
                image_patch = image[ j : (j + self.filter_size), k : (k + self.filter_size)]
                yield image_patch, j, k
                
            
    def forward_prop(self,image):
        height,width = image.shape
        conv_out = np.zeros((height - self.filter_size + 1, width - self.filter_size + 1, self.num_filters ))
        for image_patch, i, j in self.image_region(image):
            conv_out[i, j] = np.sum(image_patch*self.conv_filter, axis = (1,2))
        return conv_out
        
    def back_prop(self, dl_dout, learning_rate):
        dl_df_params = np.zeros(self.conv_filter.shape)
        for image_patch, i, j in self.image_region(self.image):
            for k in range(self.num_filters):
                dl_df_params[k] += image_patch*dl_dout[i, j, k]
                    
        
        self.conv_filter -= learning_rate*dl_df_params
        return dl_df_params
        

In [81]:
class Max_Pool:
    def __init__(self,filter_size):
        self.filter_size = filter_size
        
    def image_region(self,image):
        self.image = image
        new_height = image.shape[0]//self.filter_size
        new_width = image.shape[1]//self.filter_size
        
        
        for i in range(new_height):
            for j in range(new_width):
                image_patch = image[(i*self.filter_size) : (i*self.filter_size + self.filter_size), (j*self.filter_size) : (j*self.filter_size + self.filter_size)]
                yield image_patch, i, j
        
    def forward_prop(self,image):
        self.image = image
        height, width, num_filters = image.shape
        output = np.zeros((height//self.filter_size, width//self.filter_size ,num_filters))
     
        for image_patch, i, j in self.image_region(image):
            output[i,j] = np.amax(image_patch, axis = (0,1))
            
        return output
        
        
    def back_prop(self, dl_dout):
        dl_dmax_pool = np.zeros(self.image.shape)
        for image_patch, i, j in self.image_region(self.image):
            height, width, num_filters = image_patch.shape
            maximum_value = np.amax(image_patch, axis  = (0,1))
                
            for i1 in range(height):
                for j1 in range(width):
                    for k1 in range(num_filters):
                        if image_patch[i1, j1, k1] == maximum_value[k1]:
                            dl_dmax_pool[i*self.filter_size + i1, j*self.filter_size + j1, k1]=dl_dout[i,j,k1]
                                                    
        return dl_dmax_pool
            

In [82]:
class Softmax:
    def __init__(self, input_node, softmax_node):
        self.weight = np.random.randn(input_node, softmax_node)/input_node
        self.bias = np.zeros(softmax_node)
    
    def forward_prop(self, image):
        self.orig_im_shape = image.shape
        image_modified = image.flatten()
        self.modified_input = image_modified
        output_val = np.dot(image_modified, self.weight) + self.bias
        self.out = output_val
        exp_out = np.exp(output_val)
        return exp_out/np.sum(exp_out,axis = 0)
    
    def back_prop(self,dl_dout, learning_rate):
        for i, grad in enumerate(dl_dout):
            if grad == 0:
                continue
                
            transformation_eq = np.exp(self.out)
            s_total = np.sum(transformation_eq)
        
            dy_dz = -transformation_eq[i]*transformation_eq/(s_total**2)
            dy_dz[i] = transformation_eq[i]*(s_total-transformation_eq[i])/(s_total**2)
            
            dz_dw = self.modified_input
            dz_db = 1
            dz_d_inp = self.weight
            
            dl_dz = grad * dy_dz
            
            dl_dw = dz_dw[np.newaxis].T @ dl_dz[np.newaxis]
            dl_db = dl_dz * dz_db
            dl_d_inp = dz_d_inp @ dl_dz
            
            self.weight -= learning_rate * dl_dw
            self.bias -= learning_rate * dl_db
        
            return dl_d_inp.reshape(self.orig_im_shape)
    

In [85]:
from keras.datasets import mnist
(X_train,y_train),(X_test,y_test) = mnist.load_data()

train_images = X_train[:1500]
train_labels = y_train[:1500]

test_images = X_test[:1500]
test_labels = y_test[:1500]

conv = Conv_layer(8,3)
pool = Max_Pool(2)
softmax = Softmax(13*13*8,10)

def cnn_forward_prop(image, labels):
    out_p = conv.forward_prop((image/255) - 0.5)
    out_p = pool.forward_prop(out_p)
    out_p = softmax.forward_prop(out_p)
    
    cross_entropy = -np.log(out_p[label])
    accuracy_level = 1 if np.argmax(out_p ) == label else 0
    
    return out_p, cross_entropy, accuracy_level

def training_cnn(image, label, learn_rate=.005):
    out, loss, acc = cnn_forward_prop(image, label)
    
    gradient = np.zeros(10)
    gradient[label] = -1/out[label]
    
    grad_back = softmax.back_prop(gradient, learn_rate)
    grad_back = pool.back_prop(grad_back)
    grad_back = conv.back_prop(grad_back, learn_rate)
    
    return loss, acc

for epoch in range(10):
    shuffle_data = np.random.permutation(len(train_images))
    train_images = train_images[shuffle_data]
    train_labels = train_labels[shuffle_data]
    
    loss = 0
    num_correct = 0
    for i, (im,label) in enumerate(zip(train_images, train_labels)):
        if(i%100 == 0):
            print("%d stpes out of 100 steps: Average loss %.3f and Accuracy: %d%%" %(i+1,loss/100,num_correct))
            loss = 0
            num_correct = 0
            
        li,accuracy = training_cnn(im,label)
        loss+=li
        num_correct += accuracy
        



1 stpes out of 100 steps: Average loss 0.000 and Accuracy: 0%
101 stpes out of 100 steps: Average loss 2.270 and Accuracy: 17%
201 stpes out of 100 steps: Average loss 2.075 and Accuracy: 32%
301 stpes out of 100 steps: Average loss 1.633 and Accuracy: 54%
401 stpes out of 100 steps: Average loss 1.236 and Accuracy: 58%
501 stpes out of 100 steps: Average loss 1.001 and Accuracy: 64%
601 stpes out of 100 steps: Average loss 0.871 and Accuracy: 74%
701 stpes out of 100 steps: Average loss 0.847 and Accuracy: 76%
801 stpes out of 100 steps: Average loss 0.775 and Accuracy: 75%
901 stpes out of 100 steps: Average loss 0.754 and Accuracy: 77%
1001 stpes out of 100 steps: Average loss 0.489 and Accuracy: 88%
1101 stpes out of 100 steps: Average loss 0.691 and Accuracy: 77%
1201 stpes out of 100 steps: Average loss 0.647 and Accuracy: 79%
1301 stpes out of 100 steps: Average loss 0.726 and Accuracy: 79%
1401 stpes out of 100 steps: Average loss 0.605 and Accuracy: 77%
1 stpes out of 100 step

KeyboardInterrupt: 